# How to remove large intermediate files without breaking signatures

* **Difficulty level**: easy
* **Time need to lean**: 10 minutes or less
* **Key points**:
  * `zapped` files have only their signatures
  

### Removal of large intermediate files

SoS keep tracks of all intermediate files and will rerun steps only if any of the tracked files are removed or changed. However, it is often desired to remove some of the large non-essential intemediate files to reduce diskspace used by completed workflows, while allowing the workflow to be re-executed without these files. SoS provides a command

```
sos remove files --zap
```

to zap specified file, or for example

```
sos remove . --size +5G --zap
```
to zap all files larger than 5G. This command removes specified files but keeps a special `{file}.zapped` file with essential information (e.g. md5 signature, and size). SoS would consider a file exist when a `.zapped` file is present and will only regenerate the file if the actual file is needed for a later step.

For example, let us execute a workflow with output `temp/result.txt`, and `temp/size.txt`.

In [20]:
%run -s force
[10]
output:  "temp/result.txt"
# added comment
sh: expand=True
    dd if=/dev/urandom of={_output} count=2000

[20]
output:  'temp/size.txt'
with open(_output[0], 'w') as sz:
    sz.write(f"Modified {_input}: {os.path.getsize(_input[0])}\n")

2000+0 records in
2000+0 records out
1024000 bytes transferred in 0.082577 secs (12400549 bytes/sec)


and let us zap the intermediate file `temp/result.txt`,

In [21]:
!sos remove temp/result.txt --zap -y
!ls temp

INFO: No tracked file from 0 run are identified.
INFO: 0 file zapped
result.txt size.txt


As you can see, `temp/result.txt` is replaced with `temp/result.txt.zapped`. Now if you rerun the workflow

In [22]:
%rerun -s default -v2

INFO: default_10 (index=0) is ignored due to saved signature
INFO: default_20 (index=0) is ignored due to saved signature


## Further reading

* 